In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time
from numpy import random
random.seed(0)

In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, MiddleNode
from sequence.protocols.management.rule_manager import Rule
from sequence.protocols.entanglement.generation import EntanglementGenerationA
from sequence.protocols.entanglement.swapping import EntanglementSwappingA, EntanglementSwappingB
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

In [ ]:
def eg_rule_condition(memory_info, manager):
    if memory_info.state == "RAW":
        return [memory_info]
    else:
        return []
    
    
def eg_rule_condition21(memory_info, manager):
    if memory_info.state == "RAW" and memory_info.index < 50:
        return [memory_info]
    else:
        return []

    
def eg_rule_action12(memories_info):
    def eg_req_func(protocol):
        return isinstance(protocol, EntanglementGenerationA) and protocol.other == "r1"
    
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r2", memory, 0.85)
    protocol.primary = True
    return [protocol, ["r2"], [eg_req_func]]


def eg_rule_action21(memories_info):
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r1", memory, 0.85)
    return [protocol, [None], [None]]


def eg_rule_condition23(memory_info, manager):
    if memory_info.state == "RAW" and memory_info.index >= 50:
        return [memory_info]
    else:
        return []

    
def eg_rule_action23(memories_info):
    def eg_req_func(protocol):
        # print(protocol.memory.timeline.now(), "req_fun", protocol.name, isinstance(protocol, EntanglementGenerationA) and protocol.other == "r2")
        return isinstance(protocol, EntanglementGenerationA) and protocol.other == "r2"

    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m2", "r3", memory, 0.85)
    protocol.primary = True
    return [protocol, ["r3"], [eg_req_func]]


def eg_rule_action32(memories_info):
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m2", "r2", memory, 0.85)
    return [protocol, [None], [None]]

In [ ]:
def es_rule_conditionB(memory_info, manager):
    if memory_info.state == "ENTANGLED" and memory_info.remote_node == "r2":
        return [memory_info]
    else:
        return []
    

def es_rule_actionB(memories_info):
    memories = [info.memory for info in memories_info]
    memory = memories[0]
    protocol = EntanglementSwappingB(None, "ESB." + memory.name, memory)
    return [protocol, [None], [None]]


def es_rule_conditionA(memory_info, manager):
    if memory_info.state == "ENTANGLED":
        if memory_info.remote_node == "r1":
            for info in manager:
                if info.state == "ENTANGLED" and info.remote_node == "r3":
                    return [memory_info, info]
        elif memory_info.remote_node == "r3":
            for info in manager:
                if info.state == "ENTANGLED" and info.remote_node == "r1":
                    return [memory_info, info]
    
    return []

    
def es_rule_actionA(memories_info):
    memories = [info.memory for info in memories_info]

    def req_func1(protocol):
        return isinstance(protocol, EntanglementSwappingB) and protocol.memory.name == memories_info[0].remote_memo
    
    def req_func2(protocol):
        return isinstance(protocol, EntanglementSwappingB) and protocol.memory.name == memories_info[1].remote_memo
    
    protocol = EntanglementSwappingA(None, "ESA.%s.%s" % (memories[0].name, memories[1].name), memories[0], memories[1])
    dsts = [info.remote_node for info in memories_info]
    req_funcs = [req_func1, req_func2]
    return protocol, dsts, req_funcs

In [ ]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ns)
    qc_atten: attenuation on quantum channels (db/m)
    qc_dist: distance of quantum channels (km)
    """
    cc_delay *= 1e6
    qc_dist *= 1e3
    tl = Timeline(sim_time * 1e9)
    
    r1 = QuantumRouter("r1", tl, 50)
    r2 = QuantumRouter("r2", tl, 100)
    r3 = QuantumRouter("r3", tl, 50)
    m1 = MiddleNode("m1", tl, ["r1", "r2"])
    m2 = MiddleNode("m2", tl, ["r2", "r3"])
    
    for node1 in [r1,r2,r3,m1,m2]:
        for node2 in [r1,r2,r3,m1,m2]:
            cc = ClassicalChannel("cc_%s_%s"%(node1.name, node2.name), tl, 2e-4, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2)
    
    qc1 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
    qc1.set_ends(r1, m1)
    qc2 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
    qc2.set_ends(r2, m1)
    
    qc3 = QuantumChannel("qc_r2_m2", tl, qc_atten, qc_dist)
    qc3.set_ends(r2, m2)
    qc4 = QuantumChannel("qc_r3_m2", tl, qc_atten, qc_dist)
    qc4.set_ends(r3, m2)
    
    tl.init()
    
    # EG 1-2
    rule1 = Rule(10, eg_rule_action12, eg_rule_condition)
    r1.resource_manager.load(rule1)
    rule2 = Rule(10, eg_rule_action21, eg_rule_condition21)
    r2.resource_manager.load(rule2)
    
    # EG 2-3
    rule3 = Rule(10, eg_rule_action23, eg_rule_condition23)
    r2.resource_manager.load(rule3)
    rule4 = Rule(10, eg_rule_action32, eg_rule_condition)
    r3.resource_manager.load(rule4)
    
    # ES 1-2-3
    rule5 = Rule(10, es_rule_actionB, es_rule_conditionB)
    r1.resource_manager.load(rule5)
    rule6 = Rule(10, es_rule_actionA, es_rule_conditionA)
    r2.resource_manager.load(rule6)
    rule7 = Rule(10, es_rule_actionB, es_rule_conditionB)
    r3.resource_manager.load(rule7)
    
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    data = []
    for info in r1.resource_manager.memory_manager:
        data.append(info.entangle_time / 1e12)
    data.sort()
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("Simulation Time (s)")
    plt.ylabel("Aggregated Number of Entangled Memory")
    plt.show()
    print(data)
    
    for info in r1.resource_manager.memory_manager:
        print(info.memory.name, info.state, info.remote_memo)
    
    data = []
    for info in r2.resource_manager.memory_manager:
        data.append(info.entangle_time / 1e12)
    data.sort()
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("Simulation Time (s)")
    plt.ylabel("Aggregated Number of Entangled Memory")
    plt.show()
    print(data)
    
    data = []
    for info in r3.resource_manager.memory_manager:
        data.append(info.entangle_time / 1e12)
    data.sort()
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("Simulation Time (s)")
    plt.ylabel("Aggregated Number of Entangled Memory")
    plt.show()
    print(data)

In [ ]:
test(1000, 1000, 0.0002, 5)